In [78]:
import numpy as np
from rich.progress import track

class Tanh:
    def activate(self, x):
        return np.tanh(x)
    
    def derivative(self, x):
        return 1 - self.activate(x) ** 2


class Layer:
    def __init__(self, len_inputs, neurons, function, last=False):
        shape = neurons, len_inputs + 1
        self.weights = np.random.uniform(-0.5, 0.5, size=shape)
        self.f = function
        self.last = last
        self.idx = None
        self.neurons = neurons
        self.len_inputs = len_inputs
    
    def forward(self, layer_input):
        self.input = layer_input
        self.net = self.input.dot(self.weights.T)
        self.output = self.f.activate(self.net)
        return self.output
    
    def backward(self, alpha, previous_delta=None, previous_weigth=None, error=None):
        if self.last:
            self.delta = error * self.f.derivative(self.net)
        else:
            self.delta = (np.delete(previous_delta.dot(previous_weigth).T, 0) * self.f.derivative(self.net))
        



        self.weights += np.array([self.delta]).T * np.array([self.input]) * alpha
        
        return self.delta, self.weights
        
        
        
        
    def __repr__(self):
        return f"({self.idx}º Layer, Neurons: {self.neurons}, Last: {self.last})"


class NeuralNetwork:
    def __init__(self, *layers: Layer):
        self.layers = list(layers)
        for idx, layer in enumerate(self.layers):
            layer.idx = idx + 1
        self.layers[-1].last = True
        self.len_inputs = self.layers[0].len_inputs
        self.all_mse = []
        
    def __repr__(self):
        return f"NeuralNetwork (Num_Layers: {len(self.layers)}, Len_Inputs: {self.len_inputs}, Layers: {self.layers})"
    
    @property
    def weights(self):
        resp = []
        for idx, layer in enumerate(self.layers):
            resp.append((idx+1, layer.weights))
        return resp
        
    def _forward(self, x_input):
        #input_layer = x_input
        input_layer = np.append(1, x_input)
        for layer in self.layers:
            out_layer = layer.forward(input_layer)
            input_layer = np.append(1, out_layer)
            
        return out_layer
    
    def _backward(self, alpha, error):
        for layer in reversed(self.layers):
            if layer.last:
                previous_delta, previous_weigth = layer.backward(alpha, error=error)
            else:
                previous_delta, previous_weigth = layer.backward(alpha, previous_delta, previous_weigth)
    
    def fit(self, x_train, y_train, epochs=2000, alpha=0.05, batch_size=1):

        for epoch in track(range(epochs), description="Processing..."):
            outputs = []
            batch_errors = []
            data = list(zip(x_train,y_train))
            np.random.shuffle(data)
            x_train,y_train = zip(*data)
            x_train,y_train = np.array(x_train), np.array(y_train)
            for x, y in zip(x_train, y_train):
                out = self._forward(x)
                error = (y - out)

                batch_errors.append(error)
                if len(batch_errors) == batch_size:
                
                    batch_error = sum(batch_errors) / batch_size
                    batch_errors = []
                    self._backward(alpha, batch_error)

                outputs.append(out)
                
            errors = np.array([sum(error) for error in (y_train - outputs) ** 2])
            self.mean_squared_error = sum(errors) / len(errors)
            self.all_mse.append(self.mean_squared_error)
            
            if not epoch % 100:
                print(f"MSE: {self.mean_squared_error}")

                
                
    def predict(self, x):
        out = self._forward(x)
        return out

In [105]:
weights_1 = np.array([
    [0.2, 0.4, 0.5],
    [0.3, 0.6, 0.7],
    [0.4, 0.8, 0.3],
])
weights_2 = np.array([[-0.7, 0.6, 0.2, 0.7],[-0.3, 0.7, 0.2, 0.8]])
weights_3 = np.array([[0.1, 0.8, 0.5]])


nn = NeuralNetwork(
    Layer(2, 3, Tanh()),
    Layer(3, 2, Tanh()),
    Layer(2, 1, Tanh()),
)


nn.layers[0].weights = weights_1
nn.layers[1].weights = weights_2
nn.layers[2].weights = weights_3

x_inputs = np.array([[0.3, 0.7]])
nn._forward(x_inputs)

array([0.57636603])

In [106]:
nn._backward([[-2.0]], 0.05)
nn.predict(x_inputs)

array([0.40334683])

In [102]:
delta, w = nn.layers[2].backward(0.05, error=[-2.0])
delta, w = nn.layers[1].backward(0.05, previous_delta=delta, previous_weigth=w)
w

array([[-0.74818302,  0.5718139 ,  0.16392517,  0.66670219],
       [-0.31672278,  0.69021751,  0.18747959,  0.7884434 ]])

In [88]:
# (np.delete(previous_delta.dot(previous_weigth).T, 0) * self.f.derivative(self.net))

np.delete(delta.dot(w).T, 0) * nn.layers[1].f.derivative(nn.layers[1].net)

array([-0.96366032, -0.3344555 ])